In [21]:
import pickle
from sympy import symbols, expand, lambdify, Expr, simplify, pprint
import math
from pyomo.environ import *
import pyomo.environ as pyo
from pyomo.core.expr.numeric_expr import NumericExpression

from typing import List

In [22]:
def get_all_free_symbols(expr_list: List[Expr]):
    """
    Given a list of SymPy expressions, return a list of all unique free symbols.
    """
    all_symbols = set()
    for expr in expr_list:
        all_symbols.update(expr.free_symbols)
    return list(all_symbols)

In [23]:
def generate_constraints_from_expressions(
    expr_list: List[Expr],
    instance: ConcreteModel,
    target: float
):
    """
    Adds a ConstraintList to the model based on a list of SymPy expressions.
    Missing Pyomo scalar variables are automatically created on the model.
    Assumes constraint form: expr == 0.
    """
    instance.generated_constraints = ConstraintList()

    for expr in expr_list:
        free_syms = list(expr.free_symbols)

        pyomo_vars = []
        for sym in free_syms:
            var_name = str(sym)
            if not hasattr(instance, var_name):
                setattr(instance, var_name, Var())
            pyomo_vars.append(getattr(instance, var_name))

        # Convert SymPy expression to a function and evaluate with Pyomo vars
        expr_func = lambdify(free_syms, expr, modules=[{'exp':pyo.exp, 'pi':math.pi}, 'sympy'])
        pyomo_expr = expr_func(*pyomo_vars)
        
        # Check if pyomo_expr is already a valid Pyomo expression
        if isinstance(pyomo_expr, (NumericExpression, pyo.Expression)) or hasattr(pyomo_expr, 'is_expression_type'):
            instance.generated_constraints.add(pyomo_expr >= target)
        else:
            # Handle float result from accidentally evaluated expression
            raise TypeError(f"Expression evaluated to a constant ({pyomo_expr}) instead of a symbolic Pyomo expression.")

        # # Add the constraint (assumed to be == 0)
        # instance.generated_constraints.add(pyomo_expr >= target)

In [24]:
with open('factored_sf_exprs.pkl', 'rb') as file:
    factored_sf_exprs = pickle.load(file)

In [25]:
m = ConcreteModel()

In [26]:
generate_constraints_from_expressions(expr_list=factored_sf_exprs, instance=m, target=0.65)

In [32]:
m.generated_constraints.pprint()

generated_constraints : Size=1, Index={1}, Active=True
    Key : Lower : Body                                                                                                                                   : Upper : Active
      1 :  0.65 : 2*(-8.0*d0 + 16.0*d1 + 34.6666666666667)*exp(-10.8888888888889*(0.428571428571429*d0 - 0.857142857142857*d1 - 1)**2)/3.141592653589793 :  +Inf :   True


In [27]:
d0, d1 = symbols('d0 d1')
values = list()
for i, sf_expr in enumerate(factored_sf_exprs):
    value = sf_expr.subs({d0:4, d1:0.5})
    value = value.evalf()
    print(i, value)
    values.append(value)

target = 0.6372
matching_indices = [
    i for i, val in enumerate(values)
    if math.isclose(val, target, rel_tol=0, abs_tol=1e-4)
]
# for i in matching_indices:
#     print(f'Value at index {i} is {values[i]}')

0 2.79170360320214


In [28]:
simplify(factored_sf_exprs[0])

(-16.0*d0 + 32.0*d1 + 69.3333333333333)*exp(-10.8888888888889*(-0.428571428571429*d0 + 0.857142857142857*d1 + 1)**2)/pi

In [29]:
# # Define symbolic expression
# x, y = symbols('x y')
# expr = x**2 + 3*y
# 
# # Get free variables
# free_vars = list(expr.free_symbols)
# 
# # Create Pyomo model and variables
# model = ConcreteModel()
# sympy_to_pyomo = {}
# 
# for s in free_vars:
#     name = str(s)
#     setattr(model, name, Var())
#     sympy_to_pyomo[s] = getattr(model, name)
# 
# # Convert sympy expression to a function
# f = lambdify(free_vars, expr, modules='numpy')
# 
# # Evaluate the function using Pyomo vars
# pyomo_expr = f(*(sympy_to_pyomo[s] for s in free_vars))
# 
# # Add a constraint
# model.c1 = Constraint(expr=pyomo_expr <= 20)

In [30]:
# model.c1.pprint()